In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [2]:
data_path = '/kaggle/input/ham10000-preprocessed-dataset/balanced and standardized images meta for custom.csv'
image_folder = '/kaggle/input/ham10000-preprocessed-dataset/balanced_and_standardized_images/balanced_and_standardized_images'
df = pd.read_csv(data_path)

# Define parameters
IMG_SIZE = 96
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.5
EPOCHS = 20

# Get unique classes and create label mapping
classes = df['dx'].unique()
class_to_idx = {c: i for i, c in enumerate(classes)}
df['label'] = df['dx'].map(class_to_idx)



In [3]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.4, stratify=df['label'], random_state=42)

In [4]:
# Image Data Generators with Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of train set will be used as validation
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

Found 22529 validated image filenames belonging to 7 classes.
Found 5632 validated image filenames belonging to 7 classes.
Found 18774 validated image filenames belonging to 7 classes.


In [5]:
# Load MobileNetV2 base model
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

# Build custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(DROPOUT_RATE)(x)
output_layer = Dense(len(classes), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


705/705 ━━━━━━━━━━━━━━━━━━━━ 469s 653ms/step - accuracy: 0.2027 - loss: 2.5675 - val_accuracy: 0.3548 - val_loss: 1.7196
Epoch 2/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 241s 340ms/step - accuracy: 0.2999 - loss: 1.7663 - val_accuracy: 0.3963 - val_loss: 1.6216
Epoch 3/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 236s 333ms/step - accuracy: 0.3363 - loss: 1.6646 - val_accuracy: 0.4235 - val_loss: 1.5301
Epoch 4/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 238s 335ms/step - accuracy: 0.3590 - loss: 1.6152 - val_accuracy: 0.4446 - val_loss: 1.4859
Epoch 5/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 235s 332ms/step - accuracy: 0.3716 - loss: 1.5693 - val_accuracy: 0.4535 - val_loss: 1.4457
Epoch 6/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 236s 333ms/step - accuracy: 0.3968 - loss: 1.5228 - val_accuracy: 0.4718 - val_loss: 1.4115
Epoch 7/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 262s 333ms/step - accuracy: 0.3980 - loss: 1.5164 - val_accuracy: 0.4735 - val_loss: 1.3996
Epoch 8/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 237s 333ms/step - accuracy: 0.4136 - loss: 1.48

In [7]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], 
            name='train_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], 
            name='val_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], 
            name='train_loss'), 
        row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], 
            name='val_loss'), 
        row=1, col=2)
    fig.show()

In [8]:
plot_model_training_curve(history)

In [9]:
# Test function
def evaluate_model():
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'\nTest Accuracy: {test_acc * 100:.2f}%')

# Run evaluation
evaluate_model()

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.



587/587 ━━━━━━━━━━━━━━━━━━━━ 253s 431ms/step - accuracy: 0.4620 - loss: 1.3195

Test Accuracy: 46.41%


In [10]:
# Saving the model
model.save('/kaggle/working/MobileNetV2.keras')
model.save_weights('/kaggle/working/MobileNetV2_weights.weights.h5')